In [1]:
!pip install kafka-python avro

In [2]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [3]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [4]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [5]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [6]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [7]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [8]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [9]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [10]:
VID = "V153912"
TOKEN = "ba2d6d0056e6d9c1c63bf6de5900bdeb"

for message in txconsumer:
  # Read a transaction from the transaction topic
  txid = message.value['txid']
  payer = message.value['payer']
  payee = message.value['payee']
  amount = message.value['amount']
  print(f"Transaction: { message.value}")

  # Verify that the transaction
  signature = gen_signature(txid, payer, payee, amount, TOKEN)

  # Submit verification to submit topic
  obj = {'txid': txid, 'vid': VID, 'signature': signature}
  print(f"Submit: {obj}")
  data = serialize(submitschema, obj)
  producer.send('submit', data)

  # Get the result from the result topic
  for result in resultconsumer:
    if result.value['txid'] == txid and result.value['vid'] == VID:
      print(f"Result: {result.value}\n\n")
      break

Transaction: {'txid': 'TX02815', 'payer': 'A56629', 'payee': 'A20800', 'amount': 1320}
Submit: {'txid': 'TX02815', 'vid': 'V153912', 'signature': '196b83c7b7e827db099b344913f2bf1b'}
Result: {'timestamp': 1711639051, 'vid': 'V153912', 'txid': 'TX02815', 'checksum': '6c87555f0a9fc241480f50767e9cecf4', 'code': 200, 'message': 'Confirm'}


Transaction: {'txid': 'TX03686', 'payer': 'A09155', 'payee': 'A91080', 'amount': 1483}
Submit: {'txid': 'TX03686', 'vid': 'V153912', 'signature': 'f7b1aef35ed1ab85cebbd4e28887edf0'}
Result: {'timestamp': 1711639062, 'vid': 'V153912', 'txid': 'TX03686', 'checksum': 'cfd058f5ca30deff8ea05cb626d77d53', 'code': 200, 'message': 'Confirm'}


Transaction: {'txid': 'TX06067', 'payer': 'A89035', 'payee': 'A29232', 'amount': 976}
Submit: {'txid': 'TX06067', 'vid': 'V153912', 'signature': '00e2dbb4129245048ff816277db6e125'}
Result: {'timestamp': 1711639069, 'vid': 'V153912', 'txid': 'TX06067', 'checksum': '8bbeda7c1f320eaf0b21fe3d52b9bbdc', 'code': 200, 'message': 

KeyboardInterrupt: 